In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 函数

## 变量作用域
函数内部定义的变量、函数的形参都是局部变量，只在函数**执行时**被创建，执行完毕后销毁。

Python 解析变量时，遵循 **LEGB 规则** 来决定变量的作用域。这四个字母代表四种作用域，从 **局部（Local）** 到 **全局（Global）** 依次查找变量的定义。
- **L（Local，局部作用域）：** 函数内部定义的变量或函数参数。
- **E（Enclosing，嵌套作用域）：** 外层函数（闭包）中定义的变量。
- **G（Global，全局作用域）：** 整个脚本或模块级别的变量（不在任何函数内部定义的变量）。
- **B（Built-in，内置作用域）：** Python 语言内置的变量，比如 len()、print()、sum() 等。
### 取变量值
当 Python 解析一个变量时，会按照 LEGB 的顺序**依次查找**，找到匹配的变量**为止**，如果所有作用域都找不到该变量，就会抛出` NameError `异常。

In [ ]:
a = "Global a"  # Global作用域的变量
b = "Global b"  # Global作用域的变量
c = "Global c"  # Global作用域的变量

def func_outer():
    a = "Enclosing a"  # Enclosing作用域的变量
    b = "Enclosing b"  # Enclosing作用域的变量
    def func_inner():
        a = "Local a"  # Local作用域的变量

        # 从func_inner调用a,b,c时，a直接在Local中找到；b在Local中未找到，向上层Enclosing找，找到了；c在Local和Enclosing中均未找到，继续向上找，在Global中找到了
        print(a, b, c) 
    
    func_inner()
    print(a, b, c)

func_outer()
print(a, b, c)

Local a Enclosing b Global c
Enclosing a Enclosing b Global c
Global a Global b Global c


### 修改变量
Python 的赋值语句（=）默认不会在内层作用域中修改外层变量，而是创建新的内层局部变量：

In [ ]:
x = 10  # 全局变量

def modify_x():
    x = 20  # 创建了局部变量 x
    print("函数内 x: {}".format(x))  # ，由于局部作用域存在 x，Python 不会继续向全局作用域查找 x

modify_x()
print("函数外 x: {}".format(x))  # 仍然是 10

函数内 x: 20
函数外 x: 10


In [8]:
def outer():
    x = 10  # 外层函数的局部变量

    def inner():
        x = 20  # 这里创建了一个新的局部变量（不会修改外层 x）
        print("内部 x: {}".format(x))

    inner()
    print("外部 x: {}".format(x))  # 仍然是 10

outer()

内部 x: 20
外部 x: 10


在没有`global`或`nonlocal`指定的情况下，内层作用域禁止修改外层变量。以下情况会被解释器认为是跨域修改变量，会报`UnboundLocalError`，而不是创建新的内层局部变量：
- 赋值语句（`=`）左右两侧都有该变量名，如 `a = a + 1`
- 对该变量名使用任何赋值运算符（如 `a += 1`），即使是list也不行。

In [3]:
a = 1
def func():
    print(a)
    a = a + 1
func()

UnboundLocalError: local variable 'a' referenced before assignment

In [4]:
a = [1]
def func():
    a += [2]  # 即使list的 += 实际上会调用.extend()实现就地修改，也会被解释器认为是在跨域修改变量。
func()

UnboundLocalError: local variable 'a' referenced before assignment

在没有`global`或`nonlocal`指定的情况下，只在调用可变对象自身的就地修改方法时，才可以修改外层变量

In [5]:
a = [1]
def func():
    a.append(2)
func()
a

[1, 2]

指定了`global`或`nonlocal`后，内层作用域中的该变量名等同于外层的变量，可以用赋值语句和赋值运算符直接对该外层变量进行修改。

In [8]:
num = 1
def func():
	global num
	num = 100
	print("func内:", num)
print("global:", num)
func()
print("global:", num)

global: 1
func内: 100
global: 100


In [16]:
num = 1
def outer():
    num = 10
    def inner():
        nonlocal num   # nonlocal关键字声明
        num = 100
        print("inner", num)
    inner()
    print("outer", num)

print("global:", num)
outer()
print("global:", num)

global: 1
inner 100
outer 100
global: 1


## 参数传递
Python 的形参实质上是一个在函数执行时被创建且**指向传递进来的实参对象**的**局部变量**。
- 任何函数内对形参的**重新赋值**（可以是显式的`=`赋值语句，也可以是不可变对象的赋值运算符如`+=`）都会使该形参指向该赋值后的对象，从而与最开始的实参解绑。

In [ ]:
# 传不可变对象
def func(a):
    print("函数接收到的a的id", id(a))
    a += "World"  # str是不可变类型，没有实现__iadd__魔术方法，+=运算符实际上等同于a = a + "World"
    print("函数内修改后的a的id", id(a))  # 因为a + "World"创建了一个新对象，赋引用给a后使a与实参解绑并重新绑定。

a = "Hello"
print("函数执行前全局变量a的值", a)
print("全局变量a的id", id(a))
func(a)
print("函数执行后全局变量的a的值", a)

函数执行前全局变量a的值 Hello
全局变量a的id 2435860184240
函数接收到的a的id 2435860184240
函数内修改后的a的id 2435860182768
函数执行后全局变量的a的值 Hello


In [ ]:
# 传可变对象
def func(a):
    print("函数接收到的a的id", id(a))
    a = a + ["World"]  # 虽然list是可变类型，但这里没有使用就地修改方法而是创建了新对象后赋引用，a依然与实参解绑并重新绑定。
    print("函数内修改后的a的id", id(a))

a = ["Hello"]
print("函数执行前全局变量a的值", a)
print("全局变量a的id", id(a))
func(a)
print("函数执行后全局变量的a的值", a)

函数执行前全局变量a的值 ['Hello']
全局变量a的id 2435860190912
函数接收到的a的id 2435860190912
函数内修改后的a的id 2435860379968
函数执行后全局变量的a的值 ['Hello']


- 如果实参是一个可变对象，那么就可以通过形参调用其内部方法实现**就地修改**，修改会反映到实参原本的对象上。

In [ ]:
# 传可变对象
def func(a):
    print("函数接收到的a的id", id(a))
    a.append("World")  # 使用了可变对象的就地修改方法，没有进行重新赋值
    print("函数内修改后的a的id", id(a))

a = ["Hello"]
print("函数执行前全局变量a的值", a)
print("全局变量a的id", id(a))
func(a)
print("函数执行后全局变量的a的值", a)

函数执行前全局变量a的值 ['Hello']
全局变量a的id 2435860376448
函数接收到的a的id 2435860376448
函数内修改后的a的id 2435860376448
函数执行后全局变量的a的值 ['Hello', 'World']
